In [4]:
!cp /content/drive/MyDrive/datasets/FSDD/train_test_data.zip /content
!cp /content/drive/MyDrive/py_modules/Autoencoders/new_vae.py /content

In [5]:
import os
import pickle
from zipfile import ZipFile
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
from new_vae import Encoder, Decoder, VAE
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError 
import librosa.display

In [6]:
with ZipFile('/content/train_test_data.zip', 'r') as z:
    z.extractall()

In [7]:
x_train = np.load('/content/train_test_data/x_train.npy')
y_train = np.load('/content/train_test_data/y_train.npy')
x_test = np.load('/content/train_test_data/x_test.npy')
y_test = np.load('/content/train_test_data/y_test.npy')

In [8]:
runs = 4
lr = np.random.uniform(low=0.00001, high=0.001, size=runs)
b = np.random.randint(low=15, high=50, size=4)
eps = np.random.randint(low=70, high=120, size=runs)

In [9]:
for i in range(runs):
    print(f'eta_{i}: {lr[i]}')
    print(f'Batch_{i}: {b[i]}')
    print(f'Epochs_{i}: {eps[i]}')
    print('- '*10)

eta_0: 0.0006888396588631766
Batch_0: 17
Epochs_0: 110
- - - - - - - - - - 
eta_1: 0.0006662618826265548
Batch_1: 43
Epochs_1: 88
- - - - - - - - - - 
eta_2: 3.900668875088947e-05
Batch_2: 31
Epochs_2: 97
- - - - - - - - - - 
eta_3: 0.0002760026626469487
Batch_3: 16
Epochs_3: 86
- - - - - - - - - - 


In [10]:
encod = Encoder(
    input_shape=(256, 48, 1),
    conv_filters=(150, 100, 50, 50),
    conv_kernels=(5, 5, 5, 5),
    conv_strides=(1, 2, 2, 1),
    latent_dim=32
)
decod = Decoder(encod)
vae = VAE(encod, decod)

In [ ]:
vae.compile(optimizer=Adam(learning_rate=0.0005))
vae.fit(x_train, y_train, batch_size=32, epochs=50)

In [ ]:
save_path = '/content/drive/MyDrive/models/denoising_vae/best_model/'
vae.save_model_params(save_path)

In [15]:
latent_repr = vae.encoder(x_test)[2]
reconstruction = vae.decoder(latent_repr)
mse = MeanSquaredError()

error = mse(y_test], reconstruction).numpy()
print(error)

In [ ]:
indx = range(len(x_test))
random_indx = np.random.choice(indx, size=4)
picked = reconstruction.numpy()[random_indx]
orig = y_test[random_indx]

for rec, label in zip(picked, orig):
    print('ORIGINAL SPECTROGRAM')
    librosa.display.specshow(label[:, :, 0], hop_length=256, x_axis='time', y_axis='log')
    plt.show()

    print('RECONSTRUCTION')
    librosa.display.specshow(rec[:, :, 0], hop_length=256, x_axis='time', y_axis='log')
    plt.show()

    print('- '*10)